<a href="https://colab.research.google.com/github/shalvin-shabu/nlp-project/blob/main/nlp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets scikit-learn


In [2]:
import pandas as pd
import torch
from sklearn.metrics import f1_score
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import Dataset

In [3]:

# Load data
train_df = pd.read_csv("/content/drive/MyDrive/data/train_2kmZucJ.csv")
test_df = pd.read_csv("/content/drive/MyDrive/data/test_oJQbWVk.csv")


In [4]:
# Clean text (light)
def clean_text(text):
    import re
    text = str(text).lower()
    text = re.sub(r"http\S+|@\w+|#\w+", "", text)
    text = re.sub(r"[^\w\s$&@*#]", "", text)
    return text

train_df["clean_tweet"] = train_df["tweet"].apply(clean_text)
test_df["clean_tweet"] = test_df["tweet"].apply(clean_text)


In [5]:
# Tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

# Convert to HuggingFace dataset
train_dataset = Dataset.from_pandas(train_df[["clean_tweet", "label"]])
test_dataset = Dataset.from_pandas(test_df[["id", "clean_tweet"]])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
# Tokenize
def tokenize_function(example):
    return tokenizer(example["clean_tweet"], truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/7920 [00:00<?, ? examples/s]

Map:   0%|          | 0/1953 [00:00<?, ? examples/s]

In [7]:
# Model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Metric function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    f1 = f1_score(labels, preds, average="weighted")
    return {"weighted_f1": f1}


In [9]:
pip install --upgrade transformers


In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    do_train=True,
    do_eval=True,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    save_total_limit=1,
    save_steps=500,
    eval_steps=500
)


In [11]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset.train_test_split(test_size=0.1)["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

/tmp/ipython-input-11-2095276397.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
# Train
trainer.train()

# Predict on test set
test_predictions = trainer.predict(test_dataset)
pred_labels = test_predictions.predictions.argmax(axis=1)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shalvinshabu1 (shalvinshabu1-jyothi-engineering-college) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,0.432700
100,0.315200
150,0.291600
200,0.271800
250,0.246400
300,0.293300
350,0.258500
400,0.251000
450,0.272600
500,0.272400


In [13]:
# Evaluate on validation set
eval_result = trainer.evaluate()

# Print F1 score
print(f"\n✅ Weighted F1 Score on validation set: {eval_result['eval_weighted_f1']:.4f}")


✅ Weighted F1 Score on validation set: 0.9738


In [14]:

# Create submission
submission = pd.DataFrame({
    "id": test_df["id"],
    "label": pred_labels
})
submission.to_csv("submission.csv", index=False)
print("Submission saved as submission.csv")

Submission saved as submission.csv
